In [13]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import accelerate

In [14]:
model = SentenceTransformer('all-mpnet-base-v2')

/Users/eddiechen/Developer/Chrysalis Model/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [114]:
model = SentenceTransformer('output/checkpoint-8500')

loading configuration file output/checkpoint-8500/config.json
Model config MPNetConfig {
  "_name_or_path": "output/checkpoint-8500",
  "architectures": [
    "MPNetModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "mpnet",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "relative_attention_num_buckets": 32,
  "torch_dtype": "float32",
  "transformers_version": "4.45.1",
  "vocab_size": 30527
}

loading weights file output/checkpoint-8500/model.safetensors
All model checkpoint weights were used when initializing MPNetModel.

All the weights of MPNetModel were initialized from the model checkpoint at output/checkpoint-8500.
If your task is similar to the task the model of the checkpoint was trained on, you 

In [8]:
df = pd.read_csv('Mental Health Training.csv')
df

,s1,s2,similarity
0,"During a particularly stressful week, I made s...","When I feel stressed, I find that stepping awa...",1.00
1,"During a particularly stressful week, I made s...",I often find that I can manage my stress by ta...,0.75
2,"During a particularly stressful week, I made s...",I cope with overwhelming feelings by journalin...,0.50
3,"During a particularly stressful week, I made s...",I like to unwind by listening to music and ref...,0.25
4,"During a particularly stressful week, I made s...",I recently started a new hobby that involves p...,0.00
...,...,...,...
27397,"When I face emotional challenges, I spend time...",Nature walks greatly help me reconnect with my...,1.00
27398,I like to call my family when I'm feeling down...,Talking to family members provides me comfort ...,0.75
27399,I like to call my family when I'm feeling down...,I enjoy listening to music to distract myself.,0.50
27400,I like to call my family when I'm feeling down...,I prefer to watch cooking shows to relax.,0.25


In [116]:
# train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
# val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [117]:
# #rename similarity column
# train_df = train_df.rename(columns={'similarity': 'label'})
# val_df = val_df.rename(columns={'similarity': 'label'})
# test_df = test_df.rename(columns={'similarity': 'label'})

In [118]:
# train_df = train_df[['s1', 's2', 'label']]
# val_df = val_df[['s1', 's2', 'label']]
# test_df = test_df[['s1', 's2', 'label']]

## Pre-training evaluation on test data

In [ ]:
# test_df

In [119]:
# test_s1 = test_df['s1'].tolist()
# test_s2 = test_df['s2'].tolist()

# test_e1 = model.encode(test_s1, show_progress_bar=True)
# test_e2 = model.encode(test_s2, show_progress_bar=True)

Batches: 100%|██████████| 129/129 [00:10<00:00, 12.41it/s]


In [ ]:
# cosine_similarities = [cosine_similarity([e1], [e2])[0][0] for e1, e2 in zip(test_e1, test_e2)]
# test_df['bert_cosine_similarity'] = cosine_similarities
# test_df

In [43]:
# from sklearn.metrics import mean_squared_error
# from scipy.stats import pearsonr

# # Calculate the MSE between the BERT predictions and the actual similarity scores
# mse = mean_squared_error(test_df['label'], test_df['bert_cosine_similarity'])
# print(f"Initial Mean Squared Error: {mse}")

# # Calculate Pearson correlation between BERT similarity scores and actual similarity
# correlation, _ = pearsonr(test_df['label'], test_df['bert_cosine_similarity'])
# print(f"Initial Pearson Correlation: {correlation}")


Initial Mean Squared Error: 0.042426631650472224
Initial Pearson Correlation: 0.8203868935263074


# Finetuning SBERT

In [26]:
from sentence_transformers import (
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)

from datasets import load_dataset, Dataset
from sentence_transformers.losses import CosineSimilarityLoss

In [27]:
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)

In [28]:
print("Training", train_dataset, "Validation", val_dataset, "Testing", test_dataset)

Training Dataset({
    features: ['s1', 's2', 'label', '__index_level_0__'],
    num_rows: 38362
}) Validation Dataset({
    features: ['s1', 's2', 'label', '__index_level_0__'],
    num_rows: 8221
}) Testing Dataset({
    features: ['s1', 's2', 'label', 'bert_cosine_similarity', '__index_level_0__'],
    num_rows: 8221
})


In [29]:
train_dataset = train_dataset.remove_columns("__index_level_0__")
val_dataset = val_dataset.remove_columns("__index_level_0__")
test_dataset = test_dataset.remove_columns("__index_level_0__")
test_dataset = test_dataset.remove_columns("bert_cosine_similarity")

In [30]:
loss = CosineSimilarityLoss(model)

In [36]:
args = SentenceTransformerTrainingArguments(
    output_dir="output",
    # training params
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    warmup_ratio=0.05,
    # test params
    eval_strategy="steps",
    eval_steps=1500,
    save_total_limit=20,
    logging_strategy="steps",
    logging_steps=500,
    logging_dir="logs",
    save_strategy="steps",
    save_steps=1000,
    report_to="none",
)

In [ ]:
trainer = SentenceTransformerTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    loss=loss,
    args=args,
)

trainer.train()

In [121]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

# Calculate the MSE between the BERT predictions and the actual similarity scores
mse = mean_squared_error(test_df['label'], test_df['bert_cosine_similarity'])
print(f"Final Mean Squared Error: {mse}")

# Calculate Pearson correlation between BERT similarity scores and actual similarity
correlation, _ = pearsonr(test_df['label'], test_df['bert_cosine_similarity'])
print(f"Final Pearson Correlation: {correlation}")


Final Mean Squared Error: 0.027643172070803256
Final Pearson Correlation: 0.876991088124767


Final Mean Squared Error: 0.025065463324370946
Final Pearson Correlation: 0.8886477659645102


## Now with augmented dataset

In [6]:
backtranslated = pd.read_csv('Backtranslated Data.csv')
backtranslated.head()

,Unnamed: 0,s1,s2,similarity
0,0,"During a particularly stressful week, I made s...",I recently started a new hobby that involves l...,0.00
1,1,When I felt overwhelmed with family responsibi...,I often find it comforting to discuss my probl...,0.75
2,2,"During a particularly stressful week, I made s...",I like to relax by listening to music and thin...,0.25
3,3,"During a particularly stressful week, I made s...",I face overwhelming feelings by divulging my t...,0.50
4,4,When I felt overwhelmed with family responsibi...,"To manage family stress, I prioritized my task...",1.00


In [12]:
data = pd.concat([df, backtranslated], ignore_index=True)
data = data.drop(columns=['Unnamed: 0'])
data = data.rename(columns={'similarity': 'label'})
data = data[['s1', 's2', 'label']]
data

,s1,s2,label
0,"During a particularly stressful week, I made s...","When I feel stressed, I find that stepping awa...",1.00
1,"During a particularly stressful week, I made s...",I often find that I can manage my stress by ta...,0.75
2,"During a particularly stressful week, I made s...",I cope with overwhelming feelings by journalin...,0.50
3,"During a particularly stressful week, I made s...",I like to unwind by listening to music and ref...,0.25
4,"During a particularly stressful week, I made s...",I recently started a new hobby that involves p...,0.00
...,...,...,...
54799,"When I face emotional challenges, I spend time...",Walks in nature help me to reconnect.,1.00
54800,I like to call my family when I'm feeling down...,Talking to my family gives me comfort and stre...,0.75
54801,I like to call my family when I'm feeling down...,I'd rather watch cooking shows to relax.,0.25
54802,I like to call my family when I'm feeling down...,I like to listen to music to distract me.,0.50


In [18]:
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)

In [22]:
test_data

,s1,s2,label
39259,"When I feel overwhelmed, I take a moment to li...",I often think about my blessings to find peace...,1.00
7886,"After facing criticism for my art, I learned t...",I realized that not everyone will understand m...,0.75
49816,"When I realized I was feeling disconnected, I ...",I noticed I was alone and made plans to reconn...,1.00
47350,"When I encounter emotional difficulties, I eng...",Practice yoga helps me regain balance when I f...,0.75
842,"When I'm overwhelmed, I make a cup of herbal t...",I often brew some chamomile tea and take a mom...,1.00
...,...,...,...
24481,"When I feel anxious, I often ground myself by ...",Practicing breathing exercises helps me manage...,1.00
46612,I discovered that writing letters to myself he...,Writing letters has become a therapeutic way f...,0.50
30042,I turned my anxiety about public speaking into...,I used to worry about talking in front of grou...,1.00
10127,"When I faced anxiety before public speaking, I...",I think dogs are the best pets because they ar...,0.00


In [43]:
test_s1 = test_data['s1'].tolist()
test_s2 = test_data['s2'].tolist()

test_e1 = model.encode(test_s1, show_progress_bar=True)
test_e2 = model.encode(test_s2, show_progress_bar=True)

Batches: 100%|██████████| 257/257 [00:18<00:00, 13.92it/s]


In [44]:
cosine_similarities = [cosine_similarity([e1], [e2])[0][0] for e1, e2 in zip(test_e1, test_e2)]
test_data['bert_cosine_similarity'] = cosine_similarities
test_data

,s1,s2,label,bert_cosine_similarity
39259,"When I feel overwhelmed, I take a moment to li...",I often think about my blessings to find peace...,1.00,0.807455
7886,"After facing criticism for my art, I learned t...",I realized that not everyone will understand m...,0.75,0.697045
49816,"When I realized I was feeling disconnected, I ...",I noticed I was alone and made plans to reconn...,1.00,0.871540
47350,"When I encounter emotional difficulties, I eng...",Practice yoga helps me regain balance when I f...,0.75,0.887334
842,"When I'm overwhelmed, I make a cup of herbal t...",I often brew some chamomile tea and take a mom...,1.00,0.938282
...,...,...,...,...
24481,"When I feel anxious, I often ground myself by ...",Practicing breathing exercises helps me manage...,1.00,0.898221
46612,I discovered that writing letters to myself he...,Writing letters has become a therapeutic way f...,0.50,0.813967
30042,I turned my anxiety about public speaking into...,I used to worry about talking in front of grou...,1.00,0.722568
10127,"When I faced anxiety before public speaking, I...",I think dogs are the best pets because they ar...,0.00,0.004548


In [25]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

# Calculate the MSE between the BERT predictions and the actual similarity scores
mse = mean_squared_error(test_data['label'], test_data['bert_cosine_similarity'])
print(f"Initial Mean Squared Error: {mse}")

# Calculate Pearson correlation between BERT similarity scores and actual similarity
correlation, _ = pearsonr(test_data['label'], test_data['bert_cosine_similarity'])
print(f"Initial Pearson Correlation: {correlation}")


Initial Mean Squared Error: 0.04597581378184753
Initial Pearson Correlation: 0.8059501567230949


In [46]:
model = SentenceTransformer('output/best')

In [45]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

# Calculate the MSE between the BERT predictions and the actual similarity scores
mse = mean_squared_error(test_data['label'], test_data['bert_cosine_similarity'])
print(f"Initial Mean Squared Error: {mse}")

# Calculate Pearson correlation between BERT similarity scores and actual similarity
correlation, _ = pearsonr(test_data['label'], test_data['bert_cosine_similarity'])
print(f"Initial Pearson Correlation: {correlation}")


Initial Mean Squared Error: 0.015141007688408243
Initial Pearson Correlation: 0.9352366727141328


Metrics for augmented dataset : 

Initial Mean Squared Error: 0.04597581378184753
Initial Pearson Correlation: 0.8059501567230949

After finetuning Mean Squared Error: 0.014925692053532414
After finetuning Pearson Correlation: 0.9357486232104782
